## Concatenate all files

```bash
$ cd path/to/train-easy/
$ find -name '*.txt' -exec cat {} \; > ../../../interim/train-easy_all.txt
```

## Load libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import glob
import pickle
import json
import matplotlib.pyplot as plt
from lstm import LSTM_S2S
from metrics import exact_match_metric
from callbacks import NValidationSetsCallback, GradientLogger
from generator import DataGenerator

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Bidirectional, Layer, Input, Dense, LSTM
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
print(tf.__version__)
print("GPU Available: ", tf.test.is_gpu_available())

2.0.0-alpha0
GPU Available:  True


## Load settings

In [3]:
settings_path = Path('../../settings/settings.json')

In [4]:
with open(str(settings_path), 'r') as file:
    settings_dict = json.load(file)

In [5]:
settings_dict

{'batch_size': 1024,
 'data_path': '/storage/git/deep-math/data/raw/v1.0/',
 'epochs': 1,
 'latent_dim': 256,
 'math_module': 'numbers__round_number',
 'save_path': '/artifacts/',
 'thinking_steps': 1,
 'train_level': 'easy'}

## Load data

Start with batching a single file before tackling the whole dataset.

In [6]:
raw_path = Path(settings_dict['data_path'])
!ls {raw_path}

extrapolate  interpolate  train-easy  train-hard  train-medium


In [7]:
interpolate_path = raw_path/'interpolate'
!ls {interpolate_path} | head -5

algebra__linear_1d.txt
algebra__linear_1d_composed.txt
algebra__linear_2d.txt
algebra__linear_2d_composed.txt
algebra__polynomial_roots.txt


In [8]:
extrapolate_path = raw_path/'extrapolate'
!ls {extrapolate_path} | head -5

algebra__polynomial_roots_big.txt
arithmetic__add_or_sub_big.txt
arithmetic__add_sub_multiple_longer.txt
arithmetic__div_big.txt
arithmetic__mixed_longer.txt


In [9]:
train_easy_path = raw_path/'train-easy/'
!ls {train_easy_path} | head -5

algebra__linear_1d.txt
algebra__linear_1d_composed.txt
algebra__linear_2d.txt
algebra__linear_2d_composed.txt
algebra__polynomial_roots.txt


In [10]:
def concatenate_texts(path, pattern):
    file_paths = list(path.glob('{}*.txt'.format(pattern)))
    
    input_texts = []
    target_texts = []

    for file_path in file_paths:
        with open(str(file_path), 'r', encoding='utf-8') as f:
            lines = f.read().split('\n')[:-1]

        input_texts.extend(lines[0::2])
        target_texts.extend(['\t' + target_text + '\n' for target_text in lines[1::2]])
        
    return input_texts, target_texts

### Data settings

In [11]:
math_module = settings_dict["math_module"]
train_level = settings_dict["train_level"]

In [12]:
datasets = {
    'train':(raw_path, 'train-' + train_level + '/' + math_module),
    'interpolate':(interpolate_path, math_module),
    'extrapolate':(extrapolate_path, math_module)
           }

In [13]:
%%time

input_texts = {}
target_texts = {}

for k, v in datasets.items():
    input_texts[k], target_texts[k] = concatenate_texts(v[0], v[1])
    print('Length of set {} is {}'.format(k, len(input_texts[k])))

Length of set extrapolate is 10000
Length of set interpolate is 20000
Length of set train is 1333332
CPU times: user 568 ms, sys: 220 ms, total: 788 ms
Wall time: 800 ms


**Sample:**

In [14]:
print('INPUT:', input_texts['train'][42])
print('OUTPUT:', target_texts['train'][42].strip())

INPUT: Suppose 3*o + 31116 = 7*o. Let f(g) = 0 + o*g + 7 + 4. Let u be f(-9). Round u to the nearest one hundred thousand.
OUTPUT: -100000


Concatenate texts to get text metrics (max length, number of unique tokens, etc.):

In [15]:
all_input_texts = sum(input_texts.values(), [])
all_target_texts = sum(target_texts.values(), [])

In [16]:
input_characters = set(''.join(all_input_texts))
target_characters = set(''.join(all_target_texts))

In [17]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in all_input_texts])
max_decoder_seq_length = max([len(txt) for txt in all_target_texts])

print('Number of samples:', len(all_input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1363332
Number of unique input tokens: 51
Number of unique output tokens: 14
Max sequence length for inputs: 160
Max sequence length for outputs: 16


### Delete all texts to realease memory

In [18]:
del all_input_texts
del all_target_texts

## Create train test splits

In [19]:
input_texts_train, input_texts_valid, target_texts_train, target_texts_valid = train_test_split(input_texts['train'], target_texts['train'], test_size=0.2, random_state=42)

In [20]:
print('Number of training samples:', len(input_texts_train))

Number of training samples: 1066665


In [21]:
print('Number of validation samples:', len(input_texts_valid))

Number of validation samples: 266667


## Process text

### Vectorise the text
Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping question characters to numbers, and another for answer characters to number.

In [22]:
# Creating a mapping from unique characters to indices
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [23]:
target_token_index

{'\t': 0,
 '\n': 1,
 '-': 2,
 '.': 3,
 '0': 4,
 '1': 5,
 '2': 6,
 '3': 7,
 '4': 8,
 '5': 9,
 '6': 10,
 '7': 11,
 '8': 12,
 '9': 13}

## Create keras data generator

In [24]:
# Parameters
params = {'batch_size': settings_dict["batch_size"],
          'max_encoder_seq_length': max_encoder_seq_length,
          'max_decoder_seq_length': max_decoder_seq_length,
          'num_encoder_tokens': num_encoder_tokens,
          'num_decoder_tokens': num_decoder_tokens,
          'input_token_index': input_token_index,
          'target_token_index': target_token_index,
          'num_thinking_steps': settings_dict["thinking_steps"]
         }

In [25]:
training_generator = DataGenerator(input_texts=input_texts_train, target_texts=target_texts_train, **params)
validation_generator = DataGenerator(input_texts=input_texts_valid, target_texts=target_texts_valid, **params)
interpolate_generator = DataGenerator(input_texts=input_texts['interpolate'], target_texts=target_texts['interpolate'], **params)
extrapolate_generator = DataGenerator(input_texts=input_texts['extrapolate'], target_texts=target_texts['extrapolate'], **params)

In [26]:
example_input_batch, example_target_batch = training_generator[0][0][0], training_generator[0][0][1]
example_input_batch.shape, example_target_batch.shape

((1024, 160, 51), (1024, 17, 14))

## Train model

In [39]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.vocab_size = vocab_size
#         self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = CuDNNLSTM(self.enc_units, return_sequences=True, return_state=True)
    
    def call(self, x):
#         x = self.embedding(x)
        output, state_h, state_c = self.lstm(x)
        return output, state_h, state_c
    
    def initialize_hidden_state(self):
#         input_tensor = tf.constant(dtype=tf.float32, shape=[self.batch_sz, self.enc_units])
        return tf.zeros((self.batch_sz, self.enc_units))

    def get_placeholder(self):
        return self.lstm(Input(shape=(None, self.vocab_size)))

In [40]:
encoder = Encoder(num_encoder_tokens, 256, 0)

In [41]:
# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_h, sample_c = encoder.get_placeholder()
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (None, None, 256)
Encoder Hidden state shape: (batch size, units) (0, 256)


In [42]:
class BahdanauAttention(tf.keras.Model):

    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):

    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score

        hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)

        score = self.V(tf.nn.tanh(self.W1(values)
                       + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)

        attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)

        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return (context_vector, attention_weights)

In [43]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (None, 256)
Attention weights shape: (batch_size, sequence_length, 1) (None, None, 1)


In [44]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = CuDNNLSTM(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       )
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, _, _ = self.lstm(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, attention_weights

In [45]:
decoder = Decoder(num_decoder_tokens, 256, 256, 1024)

sample_decoder_output, _ = decoder(tf.random.uniform((1024, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (1024, 14)


In [120]:
class LstmWithAttention:
    def __init__(self, num_encoder_tokens, num_decoder_tokens, latent_dim):
        self.num_encoder_tokens = num_encoder_tokens
        self.num_decoder_tokens = num_decoder_tokens
        self.latent_dim = latent_dim

    def get_model(self):
        # Define an input sequence and process it.
        self.encoder_inputs = Input(shape=(None, self.num_encoder_tokens))
        # Use CuDNNLSTM if running on GPU
        if tf.test.is_gpu_available():
            encoder = Encoder(self.num_encoder_tokens, self.latent_dim, 1024)
        else:
            encoder = LSTM(self.latent_dim, return_state=True)
        encoder_outputs, state_h, state_c = encoder(self.encoder_inputs, self.latent_dim)
        # We discard `encoder_outputs` and only keep the states.
        self.encoder_states = [state_h, state_c]

        # Set up the decoder, using `encoder_states` as initial state.
        self.decoder_inputs = Input(shape=(None, self.num_decoder_tokens))
        # We set up our decoder to return full output sequences,
        # and to return internal states as well. We don't use the
        # return states in the training model, but we will use them in inference.
        if tf.test.is_gpu_available():
            self.decoder_lstm = CuDNNLSTM(
                self.latent_dim, return_sequences=True, return_state=True
            )
        else:
            self.decoder_lstm = LSTM(
                self.latent_dim, return_sequences=True, return_state=True
            )
        decoder_outputs, _, _ = self.decoder_lstm(
            self.decoder_inputs, initial_state=self.encoder_states
        )
        
        # Attention layer
        self.attention = BahdanauAttention(self.latent_dim)
        context_vector, attention_weights = self.attention(self.latent_dim, encoder_outputs)
#         attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])
        
        self.decoder_dense = Dense(self.num_decoder_tokens, activation="softmax")
        decoder_outputs = self.decoder_dense(decoder_outputs)

        return Model([self.encoder_inputs, self.decoder_inputs], decoder_outputs)

    def decode_sample(self, input_seq_list, target_token_index, max_sequence_length):
        # Next: inference mode (sampling).
        # Here's the drill:
        # 1) encode input and retrieve initial decoder state
        # 2) run one step of decoder with this initial state
        # and a "start of sequence" token as target.
        # Output will be the next target token
        # 3) Repeat with the current target token and current states

        # Define sampling models
        encoder_model = Model(self.encoder_inputs, self.encoder_states)

        decoder_state_input_h = Input(shape=(self.latent_dim,))
        decoder_state_input_c = Input(shape=(self.latent_dim,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_outputs, state_h, state_c = self.decoder_lstm(
            self.decoder_inputs, initial_state=decoder_states_inputs
        )
        decoder_states = [state_h, state_c]
        decoder_outputs = self.decoder_dense(decoder_outputs)

        decoder_model = Model(
            [self.decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states,
        )

        # reverse the char -> id dictionary for decoding
        reverse_target_char_index = dict(
            (i, char) for char, i in target_token_index.items()
        )

        # get the results for each sequence in the list
        results = []
        for input_seq in input_seq_list:
            results.append(
                self.decode_sequence(
                    input_seq,
                    encoder_model,
                    decoder_model,
                    target_token_index,
                    reverse_target_char_index,
                    max_sequence_length,
                )
            )
        return results

    def decode_sequence(
        self,
        input_seq,
        encoder_model,
        decoder_model,
        target_token_index,
        reverse_target_char_index,
        max_sequence_length,
    ):
        # Encode the input as state vectors.
        states_value = encoder_model.predict(input_seq)

        # Generate empty target sequence of length 1.
        target_seq = np.zeros((1, 1, self.num_decoder_tokens))
        # Populate the first character of target sequence with the start character.
        target_seq[0, 0, target_token_index["\t"]] = 1.0

        # Sampling loop for a batch of sequences
        # (to simplify, here we assume a batch of size 1).
        stop_condition = False
        decoded_sentence = ""
        while not stop_condition:
            output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

            # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_char = reverse_target_char_index[sampled_token_index]
            decoded_sentence += sampled_char

            # Exit condition: either hit max length
            # or find stop character.
            if sampled_char == "\n" or len(decoded_sentence) > max_sequence_length:
                stop_condition = True

            # Update the target sequence (of length 1).
            target_seq = np.zeros((1, 1, len(target_token_index)))
            target_seq[0, 0, sampled_token_index] = 1.0

            # Update states
            states_value = [h, c]

        return decoded_sentence

In [121]:
valid_dict = {
    'validation':validation_generator,
    'interpolation': interpolate_generator,
    'extrapolation': extrapolate_generator
}

In [122]:
history = NValidationSetsCallback(valid_dict)
gradient = GradientLogger(live_metrics=['loss', 'exact_match_metric'], live_gaps=10)

In [123]:
epochs = settings_dict['epochs']  # Number of epochs to train for.
latent_dim = settings_dict['latent_dim']  # Latent dimensionality of the encoding space.

In [124]:
lstm = LstmWithAttention(num_encoder_tokens, num_decoder_tokens, latent_dim)

In [125]:
model = lstm.get_model()

AttributeError: 'int' object has no attribute 'shape'

In [31]:
adam = Adam(lr=6e-4, beta_1=0.9, beta_2=0.995, epsilon=1e-9, decay=0.0, amsgrad=False, clipnorm=0.1)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=[exact_match_metric])
print('start training...')
train_hist = model.fit_generator(training_generator,
                                 epochs=epochs,
                                 #use_multiprocessing=True, workers=8,
                                 callbacks=[history, gradient],
                                 verbose=0,
                                )

start training...
{"chart": "live_loss", "axis": "batch"}
{"chart": "live_exact_match_metric", "axis": "batch"}
{"chart": "loss", "axis": "epoch"}
{"chart": "exact_match_metric", "axis": "epoch"}
{"chart": "live_loss", "y": 0.8754263, "x": 10}
{"chart": "live_exact_match_metric", "y": 0.0, "x": 10}
{"chart": "live_loss", "y": 0.6443325, "x": 20}
{"chart": "live_exact_match_metric", "y": 0.0, "x": 20}
{"chart": "live_loss", "y": 0.6175986, "x": 30}
{"chart": "live_exact_match_metric", "y": 0.0, "x": 30}
{"chart": "live_loss", "y": 0.6061028, "x": 40}
{"chart": "live_exact_match_metric", "y": 0.0, "x": 40}
{"chart": "live_loss", "y": 0.5812781, "x": 50}
{"chart": "live_exact_match_metric", "y": 0.0, "x": 50}
{"chart": "live_loss", "y": 0.5720814, "x": 60}
{"chart": "live_exact_match_metric", "y": 0.0, "x": 60}
{"chart": "live_loss", "y": 0.5595844, "x": 70}
{"chart": "live_exact_match_metric", "y": 0.0, "x": 70}


KeyboardInterrupt: 

In [1]:
plt.plot(train_hist.history['loss'],color='C0', label='train')
plt.plot(train_hist.history['validation_loss'], color='C0', label='valid', linestyle='--')
plt.plot(train_hist.history['extrapolation_loss'], color='C1', label='extra',)
plt.plot(train_hist.history['interpolation_loss'], color='C2', label='inter')

plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc='best')
plt.ylim([0,1])
plt.grid(True, linestyle='--')
plt.tight_layout()
plt.savefig(settings_dict['save_path'] + 'losses.png', dpi=300)

SyntaxError: invalid syntax (<ipython-input-1-aabe21b7fb28>, line 13)

In [ ]:
plt.plot(train_hist.history['exact_match_metric'],color='C0', label='train')
plt.plot(train_hist.history['validation_exact_match_metric'], color='C0', label='valid', linestyle='--')
plt.plot(train_hist.history['extrapolation_exact_match_metric'], color='C1', label='extra',)
plt.plot(train_hist.history['interpolation_exact_match_metric'], color='C2', label='inter')

plt.xlabel('epochs')
plt.ylabel('exact match metric')
plt.legend(loc='best')
plt.ylim([0,1])
plt.grid(True, linestyle='--')
plt.tight_layout()
plt.savefig(settings_dict['save_path'] + 'metrics.png', dpi=300)

In [50]:
with open(settings_dict['save_path']+'experiments_output.pkl','wb') as file:
    pickle.dump(train_hist.history, file)

In [53]:
model.save(settings_dict['save_path']+'this_model.model')

In [78]:
with open(settings_dict['save_path']+'settings.json','w') as file:
    json.dump(settings_dict, file)

FileNotFoundError: [Errno 2] No such file or directory: '../../artifacts/settings.json'